In [38]:
suppressPackageStartupMessages({
library(dplyr)
library(ggpubr)
library(ggplot2)
library(tidyverse)
library(data.table)
library(biomaRt)
     library(survival)
    library(biomaRt)
    library(survminer)
      library(GEOquery)
    })

In [2]:
fig.size <- function (height, width) {
    options(repr.plot.height = height, repr.plot.width = width)
}

In [3]:
matrix <- read.delim("GSE49711_SEQC_NB_TUC_G_log2.txt", header = T)
matrix <- matrix %>%
  gather("Sample_title",value,-c("X00gene_id"))
  
# convert from log2(FPKM + 1) to FPKM
matrix$value <- ( ( (matrix$value) ^2) - 1)

# splitting sample_titles
setDT(matrix)[, c("V1","V2","V3","V4","V5") := tstrsplit(matrix$Sample_title, "_", fixed=TRUE)]
setDT(matrix)[,"new_sample_title" := paste0(matrix$V1,"_",matrix$V2)]

matrix <-  subset(matrix, select = c("X00gene_id","new_sample_title","value"))
colnames(matrix)[colnames(matrix)=="new_sample_title"] <- "Sample_title"

In [6]:
matrix$X00gene_id %>% length

[1] 1900368

In [9]:
unique(matrix$X00gene_id) %>% length

[1] 3816

In [27]:
ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")

gene_mapping <- getBM(
    attributes = c('refseq_mrna', 'hgnc_symbol'),
    filters = 'refseq_mrna',
    values = data$RefSeqID,
    mart = ensembl
)

df <- left_join(df, gene_mapping, by = c("RefSeqID" = "refseq_mrna"))

ERROR: Error in df$RefSeqID: object of type 'closure' is not subsettable


In [28]:
gene_mapping

ERROR: Error in eval(expr, envir, enclos): object 'gene_mapping' not found


In [158]:
df %>% head

                                              
1 function (x, df1, df2, ncp, log = FALSE)    
2 {                                           
3     if (missing(ncp))                       
4         .Call(C_df, x, df1, df2, log)       
5     else .Call(C_dnf, x, df1, df2, ncp, log)
6 }                                           

### MYCN status

In [10]:
# reading in soft file to get risk and mycn status for sample_titles
family.soft <-  read.delim("GSE49711_family.soft", header = F)
# Sample_titles
sample.title <-  as.data.frame(family.soft[ (family.soft$V1 %like% "!Sample_title" ), ])
colnames(sample.title) <- "V1"
setDT(sample.title)[, c("V1", "sample_title") := tstrsplit(sample.title$V1, "=", fixed=TRUE)]

In [14]:
# high_risk
risk <-  as.data.frame(family.soft[ (family.soft$V1 %like% "!Sample_characteristics_ch1 = high risk"), ])
colnames(risk) <- "V1"
setDT(risk)[, c("V1", "high_risk") := tstrsplit(risk$V1, ":", fixed=TRUE)]

# mycn_status
mycn_status <- as.data.frame(family.soft[ (family.soft$V1 %like% "!Sample_characteristics_ch1 = mycn status"), ])
colnames(mycn_status) <- "V1"
setDT(mycn_status)[, c("V1", "mycn_status") := tstrsplit(mycn_status$V1, ":", fixed=TRUE)]



# combining sample with mycn status and risk
sample.mycn.risk <- as.data.frame(cbind(sample.title$sample_title, mycn_status$mycn_status, risk$high_risk))
colnames(sample.mycn.risk) <-  c("Sample_title","mycn_status","high_risk")
# Sample titles were of type integer, changing to character
sample.mycn.risk$Sample_title <- as.character(sample.mycn.risk$Sample_title)
# removing blank space before the sample_title
sample.mycn.risk$Sample_title <- gsub(" ","",sample.mycn.risk$Sample_title, fixed = T)

In [16]:
sample.mycn.risk %>% head

,Sample_title,mycn_status,high_risk
,<chr>,<chr>,<chr>
1,SEQC_NB001,0,1
2,SEQC_NB002,0,1
3,SEQC_NB003,1,1
4,SEQC_NB004,1,1
5,SEQC_NB005,1,1
6,SEQC_NB006,0,0


In [19]:
data <-  read.delim("GSE62564_SEQC_NB_RNA-Seq_log2RPM.txt", header = T)
data <- data %>%
  gather(variable,value, -c("RefSeqID"))

soft.file <- soft.file <- read.delim("GSE62564_family.soft", header = F)
# Sample_ID
sample.ID <-  as.data.frame(soft.file[ (soft.file$V1 %like% "!Sample_title" ), ])
colnames(sample.ID) <- "V1"
setDT(sample.ID)[, c("V1", "sample_title") := tstrsplit(sample.ID$V1, "=", fixed=TRUE)]
setDT(sample.ID)[, c("sample_ID") := tstrsplit(sample.ID$sample_title, "[2]", fixed=TRUE)]
sample.ID <- subset(sample.ID,select = c("sample_ID") )

# Sample_titles
sample.title <-  as.data.frame(soft.file[ (soft.file$V1 %like% "!Sample_geo_accession" ), ])
colnames(sample.title) <- "V1"
setDT(sample.title)[, c("V1", "sample_title") := tstrsplit(sample.title$V1, "=", fixed=TRUE)]

In [48]:
 data$RefSeqID %>% unique %>% length

[1] 43827

In [49]:
ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")

gene_mapping <- getBM(
    attributes = c('refseq_mrna', 'hgnc_symbol'),
    filters = 'refseq_mrna',
    values =  data$RefSeqID %>% unique,
    mart = ensembl
)


Batch submitting query [==>----------------------------]  11% eta: 12s

Batch submitting query [======>------------------------]  22% eta: 10s

Batch submitting query [=========>---------------------]  33% eta:  9s

Batch submitting query [=============>-----------------]  44% eta:  7s

Batch submitting query [================>--------------]  56% eta:  6s

Batch submitting query [====================>----------]  67% eta:  4s

Batch submitting query [=======================>-------]  78% eta:  3s

Batch submitting query [===========================>---]  89% eta:  1s
                                                                      



In [60]:
gene_mapping %>% dim

[1] 31965     2

In [51]:
data %>% head

,RefSeqID,variable,value
,<chr>,<chr>,<dbl>
1,NM_000014,SEQC_NB001,7.211
2,NM_000015,SEQC_NB001,-4.396
3,NM_000016,SEQC_NB001,5.144
4,NM_000017,SEQC_NB001,3.537
5,NM_000018,SEQC_NB001,6.783
6,NM_000019,SEQC_NB001,3.650


In [64]:
library(dplyr)

In [57]:
data_merged = data %>% 
  left_join(gene_mapping, by = c("RefSeqID" = "refseq_mrna")) 

Warning message in left_join(., gene_mapping, by = c(RefSeqID = "refseq_mrna")):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 324 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship = "many-to-many"` to silence this warning.”


In [62]:
data_merged %>% head

,RefSeqID,variable,value,hgnc_symbol
,<chr>,<chr>,<dbl>,<chr>
1,NM_000014,SEQC_NB001,7.211,A2M
2,NM_000015,SEQC_NB001,-4.396,NAT2
3,NM_000016,SEQC_NB001,5.144,ACADM
4,NM_000017,SEQC_NB001,3.537,ACADS
5,NM_000018,SEQC_NB001,6.783,ACADVL
6,NM_000019,SEQC_NB001,3.650,ACAT1


In [77]:
data_cleaned <- data_merged %>%
    filter(!is.na(hgnc_symbol))

In [95]:
your_tibble = data_cleaned %>%
    dplyr::select(hgnc_symbol, variable, value) %>%
    pivot_wider(names_from = variable, values_from = value, values_fn = mean) 

In [96]:
your_tibble %>% dim

[1] 18674   499

In [97]:
gene_names <- your_tibble$hgnc_symbol

In [98]:
your_tibble <- your_tibble %>% dplyr::select(-hgnc_symbol)  # Remove the GeneName column
matrix_data <- as.matrix(your_tibble)  # Convert

In [99]:
matrix_data %>% dim

[1] 18674   498

In [101]:
rownames(matrix_data) = gene_names

In [103]:
'DDIT4' %in% rownames(matrix_data)

[1] TRUE

In [36]:
gse = readRDS('/scratch/gpfs/KANG/hirak/mendel/Projects/neuroblastoma_analysis/survival/GSE62564_info.rds')

In [104]:
sample.mycn.risk %>% head

,Sample_title,mycn_status,high_risk
,<chr>,<chr>,<chr>
1,SEQC_NB001,0,1
2,SEQC_NB002,0,1
3,SEQC_NB003,1,1
4,SEQC_NB004,1,1
5,SEQC_NB005,1,1
6,SEQC_NB006,0,0


In [35]:
#efs_day 
efs.day <- as.data.frame(soft.file[ (soft.file$V1 %like% "!Sample_characteristics_ch1 = efs day" ), ])
colnames(efs.day) <- "V1"
setDT(efs.day)[, c("V1", "sample_title") := tstrsplit(efs.day$V1, "=", fixed=TRUE)]
setDT(efs.day)[, c("sample_title","EFS day") := tstrsplit(efs.day$sample_title, ":", fixed=TRUE)]
efs.day <- subset(efs.day, select = c("EFS day"))



#efs_bin
efs.bin <- as.data.frame(soft.file[ (soft.file$V1 %like% "!Sample_characteristics_ch1 = efs bin" ), ])
colnames(efs.bin) <- "V1"
setDT(efs.bin)[, c("V1", "sample_title") := tstrsplit(efs.bin$V1, "=", fixed=TRUE)]
setDT(efs.bin)[, c("sample_title","EFS bin") := tstrsplit(efs.bin$sample_title, ":", fixed=TRUE)]
efs.bin <- subset(efs.bin, select = c("EFS bin"))

#os_day
os.day <- as.data.frame(soft.file[ (soft.file$V1 %like% "!Sample_characteristics_ch1 = os day" ), ])
colnames(os.day) <- "V1"
setDT(os.day)[, c("V1", "sample_title") := tstrsplit(os.day$V1, "=", fixed=TRUE)]
setDT(os.day)[, c("sample_title","OS day") := tstrsplit(os.day$sample_title, ":", fixed=TRUE)]
os.day <- subset(os.day, select = c("OS day"))


#os_bin
os.bin <- as.data.frame(soft.file[ (soft.file$V1 %like% "!Sample_characteristics_ch1 = os bin" ), ])
colnames(os.bin) <- "V1"
setDT(os.bin)[, c("V1", "sample_title") := tstrsplit(os.bin$V1, "=", fixed=TRUE)]
setDT(os.bin)[, c("sample_title","OS bin") := tstrsplit(os.bin$sample_title, ":", fixed=TRUE)]
os.bin <- subset(os.bin, select = c("OS bin"))


# high_risk
high.risk <- as.data.frame(soft.file[ (soft.file$V1 %like% "!Sample_characteristics_ch1 = high risk:" ), ])
colnames(high.risk) <- "V1"
setDT(high.risk)[, c("V1", "sample_title") := tstrsplit(high.risk$V1, "=", fixed=TRUE)]
setDT(high.risk)[, c("sample_title","High risk") := tstrsplit(high.risk$sample_title, ":", fixed=TRUE)]
high.risk <- subset(high.risk, select = c("High risk"))

# Sex
gender <- as.data.frame(soft.file[ (soft.file$V1 %like% "!Sample_characteristics_ch1 = Sex:" ), ])
colnames(gender) <- "V1"
setDT(gender)[, c("V1", "sample_title") := tstrsplit(gender$V1, "=", fixed=TRUE)]
setDT(gender)[, c("sample_title","Gender") := tstrsplit(gender$sample_title, ":", fixed=TRUE)]
gender <- subset(gender, select = c("Gender"))

In [22]:
# training/validation:
training_validation <- as.data.frame(soft.file[ (soft.file$V1 %like% "!Sample_characteristics_ch1 = training/validation:" ), ])
colnames(training_validation) <- "V1"
setDT(training_validation)[, c("V1", "sample_title") := tstrsplit(training_validation$V1, "=", fixed=TRUE)]
setDT(training_validation)[, c("sample_title","training/validation") := tstrsplit(training_validation$sample_title, ":", fixed=TRUE)]
training_validation <- subset(training_validation, select = c("training/validation"))



survival.data <-  as.data.frame(cbind(sample.ID$sample_ID,sample.title$sample_title, efs.day$`EFS day`, efs.bin$`EFS bin`,
                                      os.day$`OS day`, os.bin$`OS bin`, high.risk$`High risk`, gender$Gender, training_validation$`training/validation`))
colnames(survival.data) <-  c("Sample ID","Sample_title","EFS day", "EFS bin","OS day","OS bin", "High Risk","Gender","training/validation")
survival.data$Sample_title <-  tolower(survival.data$Sample_title)
survival.data$Sample_title <- gsub(" ","",survival.data$Sample_title, fixed = T)
survival.data$`Sample ID` <-  as.character(survival.data$`Sample ID`)
survival.data$`Sample ID` <- gsub(" ","",survival.data$`Sample ID`, fixed = T)

survival.data <- merge(survival.data,data, by.x = "Sample ID", by.y = "variable")

Warning message in cbind(sample.ID$sample_ID, sample.title$sample_title, efs.day$`EFS day`, :
“number of rows of result is not a multiple of vector length (arg 7)”


In [23]:
survival.data %>% dim

[1] 21825846       11

In [24]:
survival.data %>% head

,Sample ID,Sample_title,EFS day,EFS bin,OS day,OS bin,High Risk,Gender,training/validation,RefSeqID,value
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,SEQC_NB001,gsm1528894,593,1,1362,1,HR,M,Training,NM_000014,7.211
2,SEQC_NB001,gsm1528894,593,1,1362,1,HR,M,Training,NM_000015,-4.396
3,SEQC_NB001,gsm1528894,593,1,1362,1,HR,M,Training,NM_000016,5.144
4,SEQC_NB001,gsm1528894,593,1,1362,1,HR,M,Training,NM_000017,3.537
5,SEQC_NB001,gsm1528894,593,1,1362,1,HR,M,Training,NM_000018,6.783
6,SEQC_NB001,gsm1528894,593,1,1362,1,HR,M,Training,NM_000019,3.650


In [106]:
survival.data$RefSeqID %>% unique %>% length

[1] 43827

In [110]:
gene_mapping %>% head

,refseq_mrna,hgnc_symbol
,<chr>,<chr>
1,NM_000014,A2M
2,NM_000015,NAT2
3,NM_000016,ACADM
4,NM_000017,ACADS
5,NM_000018,ACADVL
6,NM_000019,ACAT1


In [112]:
survival.data %>% filter(!(RefSeqID %in% gene_mapping$refseq_mrna)) %>% head

,Sample ID,Sample_title,EFS day,EFS bin,OS day,OS bin,High Risk,Gender,training/validation,RefSeqID,value
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,SEQC_NB001,gsm1528894,593,1,1362,1,HR,M,Training,NM_000029,0.041
2,SEQC_NB001,gsm1528894,593,1,1362,1,HR,M,Training,NM_000034,7.405
3,SEQC_NB001,gsm1528894,593,1,1362,1,HR,M,Training,NM_000060,1.774
4,SEQC_NB001,gsm1528894,593,1,1362,1,HR,M,Training,NM_000114,-4.396
5,SEQC_NB001,gsm1528894,593,1,1362,1,HR,M,Training,NM_000347,5.082
6,SEQC_NB001,gsm1528894,593,1,1362,1,HR,M,Training,NM_000645,4.153


In [122]:
survival_data = merge( survival.data, sample.mycn.risk , by.x = "Sample ID", by.y = "Sample_title") 

In [168]:
table(sample.mycn.risk$mycn_status)


   0    1  N/A 
 401   92    5 

In [141]:
write.csv(survival_data, 'survival_data.csv')

In [124]:
survival_data_filtered = survival_data %>% filter(RefSeqID %in% gene_mapping$refseq_mrna)

In [127]:
survival_data_filtered %>% colnames

[1] "Sample ID"           "Sample_title"        "EFS day"            
 [4] "EFS bin"             "OS day"              "OS bin"             
 [7] "High Risk"           "Gender"              "training/validation"
[10] "RefSeqID"            "value"               "mycn_status"        
[13] "high_risk"

In [129]:
ano = survival_data_filtered %>% dplyr::select(-c('RefSeqID', 'value')) %>% distinct 

In [142]:
write.csv(ano, 'survival_data.csv')

In [133]:
ano$time = round(as.numeric(ano[['OS day']], 30),0)

In [136]:
gs=c('DDIT4', 'BAIAP2', 'CDKN1A')

In [137]:
tmp = matrix_data[gs,]

In [170]:
table(ano$mycn_status)


   0    1  N/A 
 401   92    5 

## Shengling's analysis but with mycn status

In [164]:
gse <- readRDS('/scratch/gpfs/KANG/hirak/mendel/Projects/neuroblastoma_analysis/survival/GSE49710_info.rds')
fset <- fData(gse)
eset <- exprs(gse)
pset<-pData(gse)

In [161]:
vals1 = c('title','death from disease:ch1', 'high risk:ch1', 'inss stage:ch1', 'progression:ch1','Sex:ch1','tissue:ch1','age at diagnosis:ch1','mycn status:ch1')
idx = match(vals1, colnames(pset))
metadata = data.frame(pset[,idx],
                row.names = rownames(pset))

In [166]:
pset %>% colnames

[1] "title"                   "geo_accession"          
 [3] "status"                  "submission_date"        
 [5] "last_update_date"        "type"                   
 [7] "channel_count"           "source_name_ch1"        
 [9] "organism_ch1"            "characteristics_ch1"    
[11] "characteristics_ch1.1"   "characteristics_ch1.2"  
[13] "characteristics_ch1.3"   "characteristics_ch1.4"  
[15] "characteristics_ch1.5"   "characteristics_ch1.6"  
[17] "characteristics_ch1.7"   "characteristics_ch1.8"  
[19] "characteristics_ch1.9"   "treatment_protocol_ch1" 
[21] "molecule_ch1"            "extract_protocol_ch1"   
[23] "label_ch1"               "label_protocol_ch1"     
[25] "taxid_ch1"               "hyb_protocol"           
[27] "scan_protocol"           "data_processing"        
[29] "platform_id"             "contact_name"           
[31] "contact_email"           "contact_phone"          
[33] "contact_laboratory"      "contact_department"     
[35] "contact_institute"       "contact_address"        
[37] "contact_city"            "contact_zip/postal_code"
[39] "contact_country"         "supplementary_file"     
[41] "data_row_count"          "age at diagnosis:ch1"   
[43] "class label:ch1"         "dataset:ch1"            
[45] "death from disease:ch1"  "high risk:ch1"          
[47] "inss stage:ch1"          "mycn status:ch1"        
[49] "progression:ch1"         "Sex:ch1"                
[51] "tissue:ch1"

In [145]:
fdata_filtered = fData(gse) %>% dplyr::select(c("ID", "GeneSymbol")) %>%
    mutate(`GeneSymbol` = sapply(strsplit(as.character(`GeneSymbol`), " /// "), `[`, 1)) %>%
    filter(!is.na(`GeneSymbol`))

In [146]:
eset_filtered = eset[ fdata_filtered%>% rownames, ]

In [147]:
rownames(eset_filtered) = fdata_filtered$`GeneSymbol`

In [150]:
ano$Sample_title = toupper(ano$Sample_title)

In [153]:
eset_filtered[, ano$Sample_title]

ERROR: Error in eset_filtered[, ano$Sample_title]: subscript out of bounds


In [157]:
intersect(ano$Sample_title, colnames(eset_filtered)) 

character(0)

In [160]:
pset %>% head

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,⋯,age at diagnosis:ch1,class label:ch1,dataset:ch1,death from disease:ch1,high risk:ch1,inss stage:ch1,mycn status:ch1,progression:ch1,Sex:ch1,tissue:ch1
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM1205238,SEQC NB patient 001,GSM1205238,Public on Oct 21 2014,Aug 09 2013,Oct 21 2014,RNA,1,neuroblastoma,Homo sapiens,tissue: Neuroblastoma,⋯,987,1,1,1,1,4,0,1,M,Neuroblastoma
GSM1205239,SEQC NB patient 002,GSM1205239,Public on Oct 21 2014,Aug 09 2013,Oct 21 2014,RNA,1,neuroblastoma,Homo sapiens,tissue: Neuroblastoma,⋯,1808,1,2,1,1,4,0,1,M,Neuroblastoma
GSM1205240,SEQC NB patient 003,GSM1205240,Public on Oct 21 2014,Aug 09 2013,Oct 21 2014,RNA,1,neuroblastoma,Homo sapiens,tissue: Neuroblastoma,⋯,625,1,1,1,1,4,1,1,F,Neuroblastoma
GSM1205241,SEQC NB patient 004,GSM1205241,Public on Oct 21 2014,Aug 09 2013,Oct 21 2014,RNA,1,neuroblastoma,Homo sapiens,tissue: Neuroblastoma,⋯,335,N/A,2,0,1,2,1,0,F,Neuroblastoma
GSM1205242,SEQC NB patient 005,GSM1205242,Public on Oct 21 2014,Aug 09 2013,Oct 21 2014,RNA,1,neuroblastoma,Homo sapiens,tissue: Neuroblastoma,⋯,536,1,1,1,1,4,1,1,F,Neuroblastoma
GSM1205243,SEQC NB patient 006,GSM1205243,Public on Oct 21 2014,Aug 09 2013,Oct 21 2014,RNA,1,neuroblastoma,Homo sapiens,tissue: Neuroblastoma,⋯,147,0,2,0,0,4S,0,0,M,Neuroblastoma


In [169]:
table(survival_data$mycn_status)


       0        1      N/A 
17574627  4032084   219135 